### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [4]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [5]:
for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3069775455317219  /  0.07298129325778088
estimators:  10  prof:  5  | R2 mean / std:  0.3798506868379006  /  0.053844391068582506
estimators:  10  prof:  10  | R2 mean / std:  0.39138487310448683  /  0.07014887701337358
estimators:  100  prof:  1  | R2 mean / std:  0.341212460663741  /  0.037377129311028004
estimators:  100  prof:  5  | R2 mean / std:  0.4378058207159179  /  0.034852937667966426
estimators:  100  prof:  10  | R2 mean / std:  0.4224950778070913  /  0.0469991182770584
estimators:  100  prof:  1  | R2 mean / std:  0.3361904282903458  /  0.03829008607722517
estimators:  100  prof:  5  | R2 mean / std:  0.4232511439779066  /  0.04003382623409143
estimators:  100  prof:  10  | R2 mean / std:  0.41928938520859244  /  0.03806701943867207


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [6]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [7]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [8]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [9]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [10]:
grid.best_score_

0.42103851634944833

In [11]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [12]:
grid.cv_results_

{'mean_fit_time': array([0.01433905, 0.87700597, 0.01286674, 1.28846971]),
 'std_fit_time': array([0.00613737, 0.00928666, 0.00048005, 0.03294322]),
 'mean_score_time': array([0.00133761, 0.05917041, 0.00106335, 0.06278745]),
 'std_score_time': array([4.74689508e-04, 4.39519429e-03, 8.95199592e-05, 1.02020482e-03]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.35999232, 0.35290495, 0.29757566, 0.36529506]),
 'split1_test_score': array([0.43558556, 0.44873962, 0.38261396, 0.45090448]),
 'split2_test_score': array(

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [13]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [14]:
# analisando a composição do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   longitude                20640 non-null  float64
 1   latitude                 20640 non-null  float64
 2   idade_mediana_das_casas  20640 non-null  float64
 3   total_comodos            20640 non-null  float64
 4   total_quartos            20433 non-null  float64
 5   populacao                20640 non-null  float64
 6   familias                 20640 non-null  float64
 7   salario_mediano          20640 non-null  float64
 8   preco_mediano_das_casas  20640 non-null  float64
 9   proximidade_ao_mar       20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [15]:
#verificando os valores nulos
df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [16]:
# em percentual
((df.isnull().sum() / df.shape[0]) * 100).round(2)

longitude                  0.0
latitude                   0.0
idade_mediana_das_casas    0.0
total_comodos              0.0
total_quartos              1.0
populacao                  0.0
familias                   0.0
salario_mediano            0.0
preco_mediano_das_casas    0.0
proximidade_ao_mar         0.0
dtype: float64

In [17]:
# Calcular a média do total de quartos e preencher os dados faltantes com a média do total de quartos
media_total_quartos = df.total_quartos.mean()
df.total_quartos = df.total_quartos.fillna(media_total_quartos)

In [18]:
((df.isnull().sum() / df.shape[0]) * 100).round(2)

longitude                  0.0
latitude                   0.0
idade_mediana_das_casas    0.0
total_comodos              0.0
total_quartos              0.0
populacao                  0.0
familias                   0.0
salario_mediano            0.0
preco_mediano_das_casas    0.0
proximidade_ao_mar         0.0
dtype: float64

In [19]:
# separação de 25% dos dados para teste

dftrain, dftest = train_test_split(df, test_size = 0.25, random_state = 0)
print(df.shape)
print(dftrain.shape)
print(dftest.shape)

(20640, 10)
(15480, 10)
(5160, 10)


In [20]:
#preprocessamento dos dados

def preprocessamento_completo(df, dataset_de_treino = True, cat_encoder = None, std_scaler = None):

    dff = df.copy()

    
    variaveis_para_normalizar = ['latitude',
                                 'longitude',
                                 'idade_mediana_das_casas',
                                 'total_comodos',
                                 'total_quartos',
                                 'populacao',
                                 'familias',
                                 'salario_mediano']

    if dataset_de_treino:  
        
        #OHE
        encoder = OneHotEncoder()
        df_prox_mar_OHE = encoder.fit_transform(dff[['proximidade_ao_mar']]).toarray()

        #normalização
        sc = StandardScaler()
        variaveis_norm = sc.fit_transform(dff[variaveis_para_normalizar])
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y, encoder, sc
    
    else:
        #OHE
        df_prox_mar_OHE = cat_encoder.transform(dff[['proximidade_ao_mar']]).toarray()
        
        #normalização
        variaveis_norm = std_scaler.transform(dff[variaveis_para_normalizar]) 
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y

In [21]:
Xtrain, ytrain, encoder_train, scaler_train  = preprocessamento_completo(df = dftrain,
                                                                         dataset_de_treino = True, 
                                                                         cat_encoder = None, 
                                                                         std_scaler = None)

In [22]:
Xtrain.shape, ytrain.shape, dftrain.shape

((15480, 13), (15480,), (15480, 10))

In [23]:
Xtest, ytest = preprocessamento_completo(df = dftest, 
                                         dataset_de_treino = False, 
                                         cat_encoder = encoder_train, 
                                         std_scaler = scaler_train)

In [24]:
Xtest.shape, ytest.shape, dftest.shape

((5160, 13), (5160,), (5160, 10))

 > estimador_base = RandomForestRegressor()
 
 > params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [25]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [26]:
# retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [27]:
# retorna o melhor score - métrica de performance - nos dados de validação
grid.best_score_

0.7794110234821418

In [28]:
# retorna o melhor modelo, já treinado
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [29]:
# retorna uma visão geral dos resultados
grid.cv_results_

{'mean_fit_time': array([ 0.0838356 ,  8.61683623,  0.34616089, 33.07699434]),
 'std_fit_time': array([0.00598561, 0.03114257, 0.01384295, 0.53188267]),
 'mean_score_time': array([0.00205604, 0.16598511, 0.00632095, 0.50775091]),
 'std_score_time': array([9.46673553e-05, 2.36321849e-02, 4.76462734e-04, 1.12114110e-02]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.51549746, 0.52066203, 0.7651655 , 0.77856164]),
 'split1_test_score': array([0.50402416, 0.50710643, 0.75488963, 0.76927419]),
 'split2_test_score': ar

 > KNN's

In [30]:
#definindo o estimador base
estimador_base = KNeighborsRegressor()

#definindo o dicionario de parâmetros do modelo
params_KN = {"n_neighbors":[3,5]}

In [31]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_KN, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [3, 5]}, scoring='r2')

In [32]:
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [3, 5]}, scoring='r2')

In [33]:
grid.best_params_

{'n_neighbors': 5}

In [34]:
grid.best_score_

0.7082688097580078

In [35]:
grid.best_estimator_

KNeighborsRegressor()

In [36]:
grid.cv_results_

{'mean_fit_time': array([0.03698802, 0.03124563]),
 'std_fit_time': array([0.00433046, 0.00053477]),
 'mean_score_time': array([0.14699594, 0.1432093 ]),
 'std_score_time': array([0.03423861, 0.0057848 ]),
 'param_n_neighbors': masked_array(data=[3, 5],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}],
 'split0_test_score': array([0.69798389, 0.71734822]),
 'split1_test_score': array([0.6779018 , 0.69463067]),
 'split2_test_score': array([0.69109671, 0.71282754]),
 'mean_test_score': array([0.68899413, 0.70826881]),
 'std_test_score': array([0.00833219, 0.00981863]),
 'rank_test_score': array([2, 1])}

 > GradientBoostings

In [37]:
#definindo o estimador base
estimador_base = GradientBoostingRegressor()

#definindo o dicionario de parâmetros do modelo
params_GB = {"n_estimators":[1,100], "max_depth":[1,10]}

In [38]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_GB, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'max_depth': [1, 10], 'n_estimators': [1, 100]},
             scoring='r2')

In [39]:
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'max_depth': [1, 10], 'n_estimators': [1, 100]},
             scoring='r2')

In [40]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [41]:
grid.best_score_

0.8152178459960693

In [42]:
grid.best_estimator_

GradientBoostingRegressor(max_depth=10)

In [43]:
grid.cv_results_

{'mean_fit_time': array([0.00973312, 0.58169508, 0.05427837, 5.19459923]),
 'std_fit_time': array([0.00247681, 0.01351303, 0.00286312, 0.06168727]),
 'mean_score_time': array([0.00100215, 0.00300431, 0.00067234, 0.03140291]),
 'std_score_time': array([4.05233662e-07, 1.83824693e-05, 4.75490138e-04, 5.75351056e-04]),
 'param_max_depth': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1, 100, 1, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'n_estimators': 1},
  {'max_depth': 1, 'n_estimators': 100},
  {'max_depth': 10, 'n_estimators': 1},
  {'max_depth': 10, 'n_estimators': 100}],
 'split0_test_score': array([0.06022928, 0.63994132, 0.14325367, 0.81032833]),
 'split1_test_score': array([0.0580644 , 0.6258332 , 0.14087658, 0.80944029]),
 'split2_test_score': array([0.06060

----

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [143]:
class gridSearchAll():
    
    def __init__(self):
        self.grid_models = []
        
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])

        
    def fit_all(self, cv, scoring, X, y):
        for est,param in gd.grid_models:
            grid = GridSearchCV(cv=cv, estimator=est, param_grid=param,scoring=scoring)
            grid.fit(X,y)
        
    
    def best_all_grid_models(self):
        return {'Best params': grid.best_params_, 
                'Best score':grid.best_score_, 
                'Best estimator': grid.best_estimator_, 
                'Results': grid.cv_results_}

----

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [144]:
gd = gridSearchAll()

In [145]:
gd.grid_models

[]

In [146]:
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}
params_KN = {"n_neighbors":[3,5]}
params_GB = {"n_estimators":[1,100], "max_depth":[1,10]}

In [147]:
gd.grid_models

[]

In [148]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = params_KN)
gd.insert_model(estimator_base = GradientBoostingRegressor(), param_grid = params_GB)

In [149]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}],
 [KNeighborsRegressor(), {'n_neighbors': [3, 5]}],
 [GradientBoostingRegressor(),
  {'n_estimators': [1, 100], 'max_depth': [1, 10]}]]

In [150]:
gd.fit_all(3, 'r2', Xtrain, ytrain)

In [151]:
gd.best_all_grid_models()

{'Best params': {'max_depth': 10, 'n_estimators': 100},
 'Best score': 0.8148095658030723,
 'Best estimator': GradientBoostingRegressor(max_depth=10),
 'Results': {'mean_fit_time': array([0.00908287, 0.56839991, 0.05028343, 5.14320151]),
  'std_fit_time': array([0.00154293, 0.00556429, 0.00046534, 0.0537923 ]),
  'mean_score_time': array([0.00124256, 0.00311526, 0.00102584, 0.03027662]),
  'std_score_time': array([1.12600577e-03, 6.74333654e-05, 2.42471682e-05, 4.45790140e-04]),
  'param_max_depth': masked_array(data=[1, 1, 10, 10],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_n_estimators': masked_array(data=[1, 100, 1, 100],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 1, 'n_estimators': 1},
   {'max_depth': 1, 'n_estimators': 100},
   {'max_depth': 10, 'n_estimators': 1},
   {'max_depth': 10, 'n_estimators': 100}],
  'split0_test_s

----